In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import os

/home/d_nguyen11/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


Based on the files with predicted activities, we want to extract periods of continuous activities.

In [12]:
def act_grabber(ind):
  df = pd.read_csv(f'/home/d_nguyen11/Documents/reu_stats_2024/datafiles/{ind}/ml_classified_denoised_ACC_{ind}_1.csv')
  food = pd.read_csv(f'/home/d_nguyen11/Documents/reu_stats_2024/datafiles/{ind}/Food_Log_{ind}.csv')
  food['time_begin'] = pd.to_datetime(food['time_begin'])

  df['datetime'] = pd.to_datetime(df['datetime'])

  #Create folder to store activity bouts
  new_folder_name = "bouts"
  new_folder_path = os.path.join(f'/home/d_nguyen11/Documents/reu_stats_2024/datafiles/{ind}', new_folder_name)
  os.makedirs(new_folder_path, exist_ok=True)

  # Extracting continuous segments
  segments = []
  start_index = None

  for index, value in enumerate(df['activity']):
    if start_index is None and value == 1:
      start_index = index
    elif start_index is not None and value == 0:
      segments.append((start_index, index - 1))
      start_index = None

  for i, (start, end) in enumerate(segments):

    start_time = df['datetime'].iloc[start]
    end_time = df['datetime'].iloc[end]
    food_start_time = start_time - pd.Timedelta(hours = 1)
    food_end_time = end_time + pd.Timedelta(hours = 1)

    if not (food[(food["time_begin"] >= food_start_time) and (food["time_begin"] <= food_end_time)].empty):
      print("Food was detected")
    else:
      segment_df = df.iloc[start:end+1]
      if len(segment_df) > 9600: # Check if activity bouts last longer than 5 minutes
        segment_df.to_csv(f'/home/d_nguyen11/Documents/reu_stats_2024/datafiles/{ind}/{new_folder_name}/segment_{i}.csv', index=False)


In [13]:
for i in range(1,17):
  if i < 10:
    act_grabber(f'00{i}')
  else:
    act_grabber(f'0{i}')

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
df = pd.read_csv('/home/d_nguyen11/Documents/reu_stats_2024/datafiles/001/activity/segment_0.csv')
df